# T1083 - File and Directory Discovery
Adversaries may enumerate files and directories or may search in specific locations of a host or network share for certain information within a file system. Adversaries may use the information from [File and Directory Discovery](https://attack.mitre.org/techniques/T1083) during automated discovery to shape follow-on behaviors, including whether or not the adversary fully infects the target and/or attempts specific actions.

Many command shell utilities can be used to obtain this information. Examples include <code>dir</code>, <code>tree</code>, <code>ls</code>, <code>find</code>, and <code>locate</code>.(Citation: Windows Commands JPCERT) Custom tools may also be used to gather file and directory information and interact with the [Native API](https://attack.mitre.org/techniques/T1106). Adversaries may also leverage a [Network Device CLI](https://attack.mitre.org/techniques/T1059/008) on network devices to gather file and directory information (e.g. <code>dir</code>, <code>show flash</code>, and/or <code>nvram</code>).(Citation: US-CERT-TA18-106A)

## Atomic Tests

### Atomic Test #1 - File and Directory Discovery (cmd.exe)
Find or discover files on the file system.  Upon successful execution, this test will output the results of all the data discovery commands to a specified file.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
dir /s c:\ >> %temp%\T1083Test1.txt
dir /s "c:\Documents and Settings" >> %temp%\T1083Test1.txt
dir /s "c:\Program Files\" >> %temp%\T1083Test1.txt
dir "%systemdrive%\Users\*.*" >> %temp%\T1083Test1.txt
dir "%userprofile%\AppData\Roaming\Microsoft\Windows\Recent\*.*" >> %temp%\T1083Test1.txt
dir "%userprofile%\Desktop\*.*" >> %temp%\T1083Test1.txt
tree /F >> %temp%\T1083Test1.txt
```

In [ ]:
Invoke-AtomicTest T1083 -TestNumbers 1

### Atomic Test #2 - File and Directory Discovery (PowerShell)
Find or discover files on the file system. Upon execution, file and folder information will be displayed.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
ls -recurse
get-childitem -recurse
gci -recurse
```

In [ ]:
Invoke-AtomicTest T1083 -TestNumbers 2

### Atomic Test #3 - Nix File and Directory Discovery
Find or discover files on the file system

References:

http://osxdaily.com/2013/01/29/list-all-files-subdirectory-contents-recursively/

https://perishablepress.com/list-files-folders-recursively-terminal/

**Supported Platforms:** macos, linux
#### Attack Commands: Run with `sh`
```sh
ls -a >> /tmp/T1083.txt
if [ -d /Library/Preferences/ ]; then ls -la /Library/Preferences/ > /tmp/T1083.txt; fi;
file */* *>> /tmp/T1083.txt
cat /tmp/T1083.txt 2>/dev/null
find . -type f
ls -R | grep ":$" | sed -e 's/:$//' -e 's/[^-][^\/]*\//--/g' -e 's/^/ /' -e 's/-/|/'
locate *
which sh
```

In [ ]:
Invoke-AtomicTest T1083 -TestNumbers 3

### Atomic Test #4 - Nix File and Directory Discovery 2
Find or discover files on the file system

**Supported Platforms:** macos, linux
#### Attack Commands: Run with `sh`
```sh
cd $HOME && find . -print | sed -e 's;[^/]*/;|__;g;s;__|; |;g' > /tmp/T1083.txt
if [ -f /etc/mtab ]; then cat /etc/mtab >> /tmp/T1083.txt; fi;
find . -type f -iname *.pdf >> /tmp/T1083.txt
cat /tmp/T1083.txt
find . -type f -name ".*"
```

In [ ]:
Invoke-AtomicTest T1083 -TestNumbers 4

### Atomic Test #5 - Simulating MAZE Directory Enumeration
This test emulates MAZE ransomware's ability to enumerate directories using Powershell. 
Upon successful execution, this test will output the directory enumeration results to a specified file, as well as display them in the active window. 
See https://www.mandiant.com/resources/tactics-techniques-procedures-associated-with-maze-ransomware-incidents

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$folderarray = @("Desktop", "Downloads", "Documents", "AppData/Local", "AppData/Roaming")
Get-ChildItem -Path $env:homedrive -ErrorAction SilentlyContinue | Out-File -append $env:temp\T1083Test5.txt
Get-ChildItem -Path $env:programfiles -erroraction silentlycontinue | Out-File -append $env:temp\T1083Test5.txt
Get-ChildItem -Path "${env:ProgramFiles(x86)}" -erroraction silentlycontinue | Out-File -append $env:temp\T1083Test5.txt
$UsersFolder = "$env:homedrive\Users\"
foreach ($directory in Get-ChildItem -Path $UsersFolder -ErrorAction SilentlyContinue) 
{
foreach ($secondarydirectory in $folderarray)
 {Get-ChildItem -Path "$UsersFolder/$directory/$secondarydirectory" -ErrorAction SilentlyContinue | Out-File -append $env:temp\T1083Test5.txt}
}
cat $env:temp\T1083Test5.txt
```

In [ ]:
Invoke-AtomicTest T1083 -TestNumbers 5

### Atomic Test #6 - Launch DirLister Executable
Launches the DirLister executable for a short period of time and then exits.

Recently seen used by [BlackCat ransomware](https://news.sophos.com/en-us/2022/07/14/blackcat-ransomware-attacks-not-merely-a-byproduct-of-bad-luck/) to create a list of accessible directories and files.
**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: DirLister.exe must exist in the specified path #{dirlister_path}

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1083\bin\DirLister.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
$parentpath = Split-Path "PathToAtomicsFolder\T1083\bin\DirLister.exe"
Invoke-WebRequest https://github.com/SanderSade/DirLister/releases/download/v2.beta4/DirLister.v2.beta4.zip -OutFile $env:TEMP\TDirLister.v2.beta4.zip
New-Item -ItemType Directory -Force -Path $parentpath | Out-Null
Expand-Archive -Path $env:TEMP\TDirLister.v2.beta4.zip -DestinationPath $env:TEMP\TDirLister.v2.beta4 -Force
Copy-Item $env:TEMP\TDirLister.v2.beta4\* $parentpath -Recurse
Remove-Item $env:TEMP\TDirLister.v2.beta4.zip,$env:TEMP\TDirLister.v2.beta4 -Recurse -ErrorAction Ignore

```

In [ ]:
Invoke-AtomicTest T1083 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Start-Process PathToAtomicsFolder\T1083\bin\DirLister.exe
Start-Sleep -Second 4
Stop-Process -Name "DirLister"
```

In [ ]:
Invoke-AtomicTest T1083 -TestNumbers 6

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as Collection and Exfiltration, based on the information obtained.

Monitor processes and command-line arguments for actions that could be taken to gather system and network information. Remote access tools with built-in features may interact directly with the Windows API to gather information. Information may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001). Further, [Network Device CLI](https://attack.mitre.org/techniques/T1059/008) commands may also be used to gather file and directory information with built-in features native to the network device platform.  Monitor CLI activity for unexpected or unauthorized use of commands being run by non-standard users from non-standard locations.  

## Shield Active Defense
### Decoy Content  
 Seed content that can be used to lead an adversary in a specific direction, entice a behavior, etc. 

 Decoy Content is the data used to tell a story to an adversary. This content can be legitimate or synthetic data which is used to reinforce or validate your defensive strategy. Examples of decoy content are files on a storage object, entries in the system registry, system shortcuts, etc.
#### Opportunity
There is an opportunity to feed content to an adversary to influence their behaviors, test their interest in specific topics, or add legitimacy to a system or environment.
#### Use Case
A defender can utilize decoy files and directories to provide content that could be used by the adversary.
#### Procedures
Create directories and files with names and contents using key words that may be relevant to an adversary to see if they examine or exfiltrate the data.
Seed a file system with content that is of no value to the company but reinforces the legitimacy of the system if viewed by an adversary.